In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create Spark session
spark = SparkSession.builder.appName("SMS Spam Classifier Without IDF").getOrCreate()

# Load dataset
data = spark.read.csv("SMSSpamCollection", sep="\t", inferSchema=True).toDF("label", "message")

# Convert 'spam'/'ham' to numeric labels: spam = 1.0, ham = 0.0
from pyspark.sql.functions import when
data = data.withColumn("label", when(data.label == "spam", 1.0).otherwise(0.0))

# Split into train and test sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Tokenize text
tokenizer = Tokenizer(inputCol="message", outputCol="words")
train_words = tokenizer.transform(train_data)
test_words = tokenizer.transform(test_data)

# Convert words to term frequency features using HashingTF
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=1000)
train_tf = hashingTF.transform(train_words).select("label", "features")
test_tf = hashingTF.transform(test_words).select("label", "features")

# Train Naive Bayes model (Multinomial model requires non-negative features)
nb = NaiveBayes(modelType="multinomial")
model = nb.fit(train_tf)

# Make predictions
predictions = model.transform(test_tf)

# Evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy (Without IDF): {accuracy:.2f}")


25/04/10 23:29:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/04/10 23:29:12 ERROR Executor: Exception in task 1.0 in stage 14.0 (TID 24)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`Tokenizer$$Lambda$3150/0x0000000801427840`: (string) => array<string>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.ap

Py4JJavaError: An error occurred while calling o529.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 14.0 failed 1 times, most recent failure: Lost task 1.0 in stage 14.0 (TID 24) (192.168.1.195 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`Tokenizer$$Lambda$3150/0x0000000801427840`: (string) => array<string>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext